In [ ]:
import pandas as pd
import sys

sys.path.append("../")
from toolkit.helpers.df_functions import fix_null_ints
from toolkit.compare_case_groups.temporal_process import (
    build_temporal_data,
    create_window_df,
)
import polars as pl

from toolkit.compare_case_groups.build_dataframes import (
    build_attribute_df,
    build_grouped_df,
    build_ranked_df,
    filter_df,
)


case_groups_model_df_pd = pd.read_csv(
    "../example_outputs/compare_case_groups/customer_complaints_prepared.csv"
)
case_groups_model_df_pd["Subject ID"] = [
    str(x) for x in range(1, len(case_groups_model_df_pd) + 1)
]
filters = ""
groups = ["name", "age", "city"]
aggregates = ["delivery_issue"]
temporal = ""

case_groups_model_df_pd = case_groups_model_df_pd.replace("", None)

case_groups_model_df_pd = fix_null_ints(case_groups_model_df_pd)

filtered_df = filter_df(case_groups_model_df_pd, filters)

# create group df
grouped_df = build_grouped_df(filtered_df, groups)

# create attribute df
filtered_df_pl = pl.from_pandas(filtered_df)
attributes_df = build_attribute_df(filtered_df_pl, groups, aggregates=aggregates)

temporal_df = pl.DataFrame()
temporal_atts = []
# create Window df
if temporal != "":
    temporal_df = create_window_df(
        groups, temporal, aggregates, pl.from_pandas(filtered_df)
    )

    temporal_atts = sorted(case_groups_model_df_pd[temporal].astype(str).unique())

    temporal_df = build_temporal_data(temporal_df, groups, temporal_atts, temporal)
# Create overall df
odf = build_ranked_df(
    temporal_df,
    pl.from_pandas(grouped_df),
    attributes_df,
    temporal,
    groups,
)
print(len(odf))

In [ ]:
odf.head()

In [ ]:
groups_text = f"[{", ".join(["**" + g + "**" for g in groups])}]"
description = "This table shows:"
description += f"A summary of all **{len(case_groups_model_df_pd)}** data records"
description += f"\n- The **Group Count** of records for all {groups_text} groups, and corresponding **Group Rank**"
description += f"\n- The **Attribute Count** of each **Attribute Value** for all {groups_text} groups, and corresponding **Attribute Rank**"
print(description)